# Listing all topologies for a finite set

Consider a a finte $n$-elements set 
$$X=\{0,1,\ldots,n-1\}.$$
We can encode a subset $S \subset 2^X$ as a binary tuple ("binary mask") of length $n$: 
$$S\simeq(b_0, b_1,\ldots, b_n),\quad b_i\in\{0,1\},$$
and $b_i=1$ whenever $i\in S$.

For three subsets $S^j\simeq(b^j_i)_i$, $j=1,2,3$ we have
\begin{equation}
    \begin{aligned}
    S^3 &= S^1 \bigcap S^2 \Leftrightarrow b^3_i=b_i^1 b_i^2,\\
    S^3 &= S^1 \bigcup S^2 \Leftrightarrow b^3_i=b_i^1 + b_i^2 - b_i^1 b_i^2.
    \end{aligned}
\end{equation}

A set of tuples $T\in  2^{\{0,1\}^n}\simeq 2^{2^X}$ can be considered as providing topology for $X$, whenever
1. $(1,\ldots,1)\in T$ and  $(0,\ldots,0)\in T$ ($X$ and $\emptyset$ are inside);
2. For any $b^1, b^2\in T$: $(b^1+b^2-b^1b^2)\in T$ (this correponds to a union of any two subsets);
3. For any $b^1, b^2\in T$: $b^1b^2\in T$ (this correponds to an intersection of any two subsets).

All operations with tuples are considered in a "point-wise" fashion, like in Eq.(1). Note that the second point is valid due to finitness of $X$, otherwise one should consider arbitrary uniouns.

Next, we make a code to list all the topologies for $n$-elements sets via a exhaustive check over all $2^{2^{n}}$ candidates.

In [32]:
def join_subsets(subset1: tuple, subset2: tuple) -> tuple:
    """Makes a union of two subsets represented as tuples"""
    assert len(subset1) == len(subset2)
    assert (
        max(subset1) <= 1
        and max(subset2) <= 1
        and min(subset1) >= 0
        and min(subset2) >= 0
    )
    set_rslt = []
    for a, b in zip(subset1, subset2):
        set_rslt.append(a + b - a * b)
    return tuple(set_rslt)


def intersect_subsets(subset1: tuple, subset2: tuple) -> tuple:
    """Makes an itersection of two subsets represented as tuples"""
    assert len(subset1) == len(subset2)
    assert (
        max(subset1) <= 1
        and max(subset2) <= 1
        and min(subset1) >= 0
        and min(subset2) >= 0
    )
    set_rslt = []
    for a, b in zip(subset1, subset2):
        set_rslt.append(a * b)
    return tuple(set_rslt)


def make_set_of_subsets_from_bin_mask(bin_mask: list, n: int) -> set:
    """Reconstructs a set of subsets (tuples) from a binary tuple of 2^n length for given n.
    For example, for n=2, (1,0,0,1) is converted to the antidiscrete topology:
    1 (0,0) empty set is inside
    0 (0,1) {1} is not included
    0 (1,0) {0} is not included
    1 (1,1) full set {0,1} is inside
    """
    assert len(bin_mask) == 2**n
    set_of_subsets = set()
    for ind in range(2**n):
        if bin_mask[ind] == 1:
            subset = tuple([int(dit) for dit in bin(ind)[2:].zfill(n)])
            set_of_subsets.add(subset)
    return set_of_subsets


def make_bin_mask_from_int(ind: int, n: int) -> tuple[int]:
    """Prepares a 'binary mask' of length 2^n from an 'index' of this mask (from 0 to 2^(2^n)-1).
    Goes as input for make_set_of_subsets_from_bin_mask(bin_mask, n)
    Example: for n = 2
    ind = 0 -> (0, 0, 0, 0)
    ind = 1 -> (0, 0, 0, 1)
    ind = 2 -> (0, 0, 1, 0)
    ...
    ind = 15 -> (1, 1, 1, 1)
    """
    assert 0 <= ind < 2 ** (2**n)
    bin_mask = tuple([int(dit) for dit in bin(ind)[2:].zfill(2**n)])
    return bin_mask


def convert_subset_to_familiar_form(subset: tuple) -> tuple:
    """Converts subset in the form of a binary tuple to a tuple with weight(sebset) elements
    with particular elements.
    Example: (1, 0, 1, 1, 0) is converted to frozenset {0, 2, 4}.
    """
    assert max(subset) <= 1 and min(subset) >= 0
    n = len(subset)
    set_rslt = []
    for i in range(n):
        if subset[i] == 1:
            set_rslt.append(i)
    return frozenset(set_rslt)
    #return tuple(set_rslt)


def convert_set_of_subsets_to_familiar_topology(
    set_of_subsets: set[tuple],
) -> set[tuple]:
    """Converts sets-of-subsets in the form like {(0, 0, 0), (1, 1, 1)}
    to a more readable form {(), (0, 1, 2)}"""
    topology = set()
    for subset in set_of_subsets:
        topology.add(convert_subset_to_familiar_form(subset))
    return topology

In [33]:
# *** Playground ***
n = 2
ind = 10
print(
    f"{ind=} for {n=} corresponds to a mask of a subset of set's subsets {make_bin_mask_from_int(ind, n)}"
)

bin_mask = (1, 1, 0, 1)
print(
    f"For a mask {bin_mask}  and {n=} we have set-of-subsets: {make_set_of_subsets_from_bin_mask(bin_mask, n=2)}"
)

subset = (1, 1)
print(
    f"Subset encoded in {subset} is actually {set(convert_subset_to_familiar_form(subset))}"
)

ind=10 for n=2 corresponds to a mask of a subset of set's subsets (1, 0, 1, 0)
For a mask (1, 1, 0, 1)  and n=2 we have set-of-subsets: {(0, 1), (1, 1), (0, 0)}
Subset encoded in (1, 1) is actually {0, 1}


In [34]:
def check_topolginess(set_of_subsets: set, n: int = 3, verbose: bool = False) -> bool:
    """Checks whether a given set-of-subsets provides a topology for given n"""
    # Check that the whole set is in the set_of_subsets
    if tuple([1] * n) not in set_of_subsets:
        if verbose:
            print(f"ERROR: Full set {tuple([1] * n)} is not in the given set of subsets")
        return False

    # Check that the empty set is in the set_of_subsets
    if tuple([0] * n) not in set_of_subsets:
        if verbose:
            print(f"ERROR: Empty set {tuple([1] * n)} is not in the given set of subsets")
        return False

    for ind, subset1 in enumerate(set_of_subsets):
        for ind, subset2 in list(enumerate(set_of_subsets))[ind + 1 :]:
            # Check that union is in the set_of_subsets
            if join_subsets(subset1, subset2) not in set_of_subsets:
                if verbose:
                    print(
                        f"ERROR:: Join of {subset1} and {subset2} is not in  the given set of subsets"
                    )
                return False
            # Check that intersection is in the set_of_subsets
            if intersect_subsets(subset1, subset2) not in set_of_subsets:
                if verbose:
                    print(
                        f"ERROR: Intersection of {subset1} and {subset2} is not in  the given set of subsets"
                    )
                return False
    return True

In [35]:
# *** Playground ***
# Sanite check for the antidiscrete topology
set_of_subsets = {(1, 1, 1), (0, 0, 0)}
print(check_topolginess(set_of_subsets, 3))

# Sanite check for some arbitrary set of subsets
set_of_subsets = {(1, 1, 1), (0, 1, 0)}
print(check_topolginess(set_of_subsets, 3, verbose=True))

True
ERROR: Empty set (1, 1, 1) is not in the given set of subsets
False


In [36]:
def print_topology(topology: set) -> None:
    """Printing topology in a pretty way"""
    N = len(topology)
    output = "{"
    for i, subset in enumerate(topology):
        subset_str = str(set(subset)) if len(subset) > 0 else '{}'
        output += subset_str
        if i < N - 1:
            output += ', '
        else:
            output += '}'
    print(output)

In [37]:
n = 2

list_of_topologies = []
for ind in range(2 ** (2**n)):
    bin_mask = make_bin_mask_from_int(ind, n)
    set_of_subsets = make_set_of_subsets_from_bin_mask(bin_mask, n=n)
    if check_topolginess(set_of_subsets, n=n, verbose=False):
        topology = convert_set_of_subsets_to_familiar_topology(set_of_subsets)
        list_of_topologies.append(topology)
print(f"For {n=}, {len(list_of_topologies)} topologies in total")
print(f"Explicit list of all topologies for a set {set(range(n))}:")
for topology in list_of_topologies:
    print_topology(topology)

For n=2, 4 topologies in total
Explicit list of all topologies for a set {0, 1}:
{{}, {0, 1}}
{{}, {0, 1}, {0}}
{{}, {0, 1}, {1}}
{{}, {0, 1}, {1}, {0}}
